In [1]:
## import tweet data
import pandas as pd
import os
string = "../dat/musk_tweets/"
df_tweets = pd.read_csv(string+os.listdir(string)[0])
for f in os.listdir(string)[1:]:
    df_tweet = pd.read_csv(string+f)
    df_tweets = df_tweets.append(df_tweet)
    
df_tweets = df_tweets.drop("Unnamed: 0", axis = 1)

index = df_tweets.tweet.str.contains("dogecoin", case=False)
df_tweets[index].tweet

2355         @28delayslater @justpaulinelol Dogecoin Mode
5597     Dogecoin value may vary  https://t.co/UWerAhPv63
5598               Dogecoin rulz  https://t.co/flWWUgAgLU
5599                                      @dogecoin Uh oh
5600    @Tom_Heats Dogecoin might be my fav cryptocurr...
2192     Dogecoin value may vary  https://t.co/UWerAhPv63
2193               Dogecoin rulz  https://t.co/flWWUgAgLU
2194                                      @dogecoin Uh oh
2195    @Tom_Heats Dogecoin might be my fav cryptocurr...
58      Origin of Doge Day Afternoon: The ancient Roma...
149                    @draecomino 2022: Dogecoin is dumb
221     @PPathole @dogecoin Just some used Antminer L3...
223     @dogecoin I just set up some little Doge minin...
254     If major Dogecoin holders sell most of their c...
276     Bought some Dogecoin for lil X, so he can be a...
310                         @Jbuttermost @Dogecoinrich 🤣🤣
312                                      @Dogecoinrich 🤣🤣
351           

Change date column to datetime format

In [2]:
df_tweets.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,416314131545739264,416265844633899008,1.388092e+12,2013-12-26 21:06:38,0,NaN,"@olajahr @TeslaMotors Ideally, the S should ju...",en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,415949158127005698,415904867681460224,1.388005e+12,2013-12-25 20:56:21,0,NaN,@bonnienorman @TeslaMotors @TeslaMotorsClub Me...,et,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,414560643983740928,414456595431432192,1.387674e+12,2013-12-22 00:58:54,0,NaN,"@simonhackett Yes, definitely needs a huge amo...",en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,414456595431432192,414456595431432192,1.387649e+12,2013-12-21 18:05:27,0,NaN,This is why I think solar power will be the pr...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,414456263087370240,414387939661979649,1.387649e+12,2013-12-21 18:04:08,0,NaN,@arcaresenal true,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [3]:
## Change date to datetime format for merging
from datetime import datetime
import numpy as np

date_changer = lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
new_date =  np.array([date_changer(xi) for xi in df_tweets["date"]])

df_tweets["date"] = new_date
df_tweets["date"] = df_tweets["date"].dt.date
df_tweets["date"] = df_tweets["date"].astype("string")
df_tweets["Date"] = df_tweets["date"]

In [4]:
##filter for dogecoin tweets
index = df_tweets.tweet.str.contains("dogecoin", case=False)
df_tweets = df_tweets[index]

hot_tweets = df_tweets.date 
hot_tweets = pd.to_datetime(hot_tweets)

In [5]:
## extract hot tweets 
hot_tweets

2355   2020-04-25
5597   2019-04-02
5598   2019-04-02
5599   2019-04-02
5600   2019-04-02
2192   2019-04-02
2193   2019-04-02
2194   2019-04-02
2195   2019-04-02
58     2021-03-13
149    2021-03-02
221    2021-02-20
223    2021-02-20
254    2021-02-14
276    2021-02-10
310    2021-02-07
312    2021-02-07
351    2021-02-04
2977   2020-04-25
6219   2019-04-02
6220   2019-04-02
6221   2019-04-02
6222   2019-04-02
Name: date, dtype: datetime64[ns]

In [6]:
## create dogcoin data
df_dog = pd.read_csv("../dat/crypto-stocks/doge-coin-daily.csv")

In [7]:
## change date column to datetime
date_changer = lambda x: datetime.strptime(x,"%Y-%m-%d")
new_date =  np.array([date_changer(xi) for xi in df_dog["Date"]])

df_dog["Date"] = new_date 
df_dog["Date"] = df_dog["Date"].astype("string")


In [8]:
## merge the two datasets 
merged_data = df_tweets.merge(df_dog,how = "outer" ,on = "Date")
merged_data = merged_data.drop(labels = "date", axis = 1)
merged_data.columns

Index(['id', 'conversation_id', 'created_at', 'timezone', 'place', 'tweet',
       'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'],
      dtype='object')

In [9]:
## sort values by Date
merged_data["Date"] = pd.to_datetime(merged_data["Date"])
merged_data = merged_data.sort_values(by="Date")

In [10]:
## sanity check
merged_data[~ merged_data.tweet.isnull()]

,id,conversation_id,created_at,timezone,place,tweet,language,hashtags,cashtags,user_id,...,retweet_date,translate,trans_src,trans_dest,Date,Open,High,Low,Close,Volume
3,1.113173e+18,1.113173e+18,1.554236e+12,0.0,NaN,Dogecoin rulz https://t.co/flWWUgAgLU,ht,[],[],44196397.0,...,NaN,NaN,NaN,NaN,2019-04-02,0.002459,0.002863,0.002394,0.002795,60298362
2,1.113179e+18,1.113179e+18,1.554238e+12,0.0,NaN,Dogecoin value may vary https://t.co/UWerAhPv63,en,[],[],44196397.0,...,NaN,NaN,NaN,NaN,2019-04-02,0.002459,0.002863,0.002394,0.002795,60298362
4,1.113164e+18,1.113146e+18,1.554234e+12,0.0,NaN,@dogecoin Uh oh,en,[],[],44196397.0,...,NaN,NaN,NaN,NaN,2019-04-02,0.002459,0.002863,0.002394,0.002795,60298362
5,1.113009e+18,1.113009e+18,1.554197e+12,0.0,NaN,@Tom_Heats Dogecoin might be my fav cryptocurr...,en,[],[],44196397.0,...,NaN,NaN,NaN,NaN,2019-04-02,0.002459,0.002863,0.002394,0.002795,60298362
6,1.113179e+18,1.113179e+18,1.554238e+12,0.0,NaN,Dogecoin value may vary https://t.co/UWerAhPv63,en,[],[],44196397.0,...,NaN,NaN,NaN,NaN,2019-04-02,0.002459,0.002863,0.002394,0.002795,60298362
7,1.113173e+18,1.113173e+18,1.554236e+12,0.0,NaN,Dogecoin rulz https://t.co/flWWUgAgLU,ht,[],[],44196397.0,...,NaN,NaN,NaN,NaN,2019-04-02,0.002459,0.002863,0.002394,0.002795,60298362
13,1.113009e+18,1.113009e+18,1.554197e+12,0.0,NaN,@Tom_Heats Dogecoin might be my fav cryptocurr...,en,[],[],44196397.0,...,NaN,NaN,NaN,NaN,2019-04-02,0.002459,0.002863,0.002394,0.002795,60298362
12,1.113164e+18,1.113146e+18,1.554234e+12,0.0,NaN,@dogecoin Uh oh,en,[],[],44196397.0,...,NaN,NaN,NaN,NaN,2019-04-02,0.002459,0.002863,0.002394,0.002795,60298362
11,1.113173e+18,1.113173e+18,1.554236e+12,0.0,NaN,Dogecoin rulz https://t.co/flWWUgAgLU,ht,[],[],44196397.0,...,NaN,NaN,NaN,NaN,2019-04-02,0.002459,0.002863,0.002394,0.002795,60298362
10,1.113179e+18,1.113179e+18,1.554238e+12,0.0,NaN,Dogecoin value may vary https://t.co/UWerAhPv63,en,[],[],44196397.0,...,NaN,NaN,NaN,NaN,2019-04-02,0.002459,0.002863,0.002394,0.002795,60298362


In [11]:
## create full dataset to work with usefull columns
full_dataset = merged_data[["tweet",'hashtags', 'cashtags', 
                                'day', 'hour', 'link', 'urls', 
                                'retweet', 'nlikes', 'nreplies', 
                                'nretweets', 'quote_url','reply_to', 
                                'retweet_date','Date', 'Open', 'High', 
                                'Low', 'Close', 'Volume']]



In [25]:
#full_dataset[full_dataset["Date"] == "2019-04-02"]
hot_day_ind = np.where(full_dataset["Date"] == "2021-02-04")[0]
start = hot_day_ind[-1]


data_first_tweet = full_dataset[start-30:start+30]

In [26]:
##Example plot of first hot tweet
import plotly.express as px
fig = px.line(data_first_tweet, x="Date", y=["Low","High"], title='DogeCoin Price')
fig.show()

In [14]:
## save dataset
full_dataset.to_csv('../dat/cleaned_data/hot_tweets_merged_dataset.csv')